In [9]:
from parameters import FileParams as fp
import os
import numpy as np

In [17]:
"""
Load training data
"""
train_x = np.load(os.path.join(fp.data_dir, 'train_x.npy'))
train_y = np.load(os.path.join(fp.data_dir, 'train_y.npy'))
print(f'Percent of data positive: {np.mean(train_y)}')
print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
print(len(train_x))

Percent of data positive: 0.1363780545464339
train_x size:  480.5892868041992 MB
167087


In [24]:
"""
Load validation data
"""
val_x = np.load(os.path.join(fp.data_dir, 'val_x.npy'))
val_y = np.load(os.path.join(fp.data_dir, 'val_y.npy'))
print(f'Percent of data positive: {np.mean(val_y)}')
print('val_y size: ', val_x.nbytes / 1024 / 1024, 'MB')

Percent of data positive: 0.00011156978690170702
val_y size:  25.780113220214844 MB


In [26]:
# Train the model
from model import get_model
from keras.callbacks import TensorBoard, ModelCheckpoint

BATCH_SIZE = 128
EPOCHS = 10

model_checkpoint_callback = ModelCheckpoint(
    filepath='./checkpoints/model1',
    save_weights_only=False,  # Save the whole model
    monitor='val_loss',
    mode='min',
    save_best_only=True)

my_model = get_model()
my_model.summary()
my_model.fit(
    train_x, 
    train_y, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    validation_data=(val_x, val_y),
    callbacks=[TensorBoard(log_dir=fp.log_dir)]
    )

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_5 (GRU)                 (None, 20)                2100      
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
  24/1306 [..............................] - ETA: 47s - loss: 0.3709 - accuracy: 0.8271

KeyboardInterrupt: 

In [16]:
# Save the model
my_model.save(os.path.join('models', 'model.net'))

INFO:tensorflow:Assets written to: models\model.net\assets


In [8]:
# Save the test data
np.save(os.path.join(fp.data_dir, 'test_x.npy'), test_x)
np.save(os.path.join(fp.data_dir, 'test_y.npy'), test_y)

In [ ]:
# Load the test data
test_x = np.load(os.path.join(fp.data_dir, 'test_x.npy'))
test_y = np.load(os.path.join(fp.data_dir, 'test_y.npy'))

In [27]:
# Evaluate the model
loss, accuracy = my_model.evaluate(test_x, test_y)
print('Test loss: ', loss)
print('Test accuracy: ', accuracy*100, '%')

NameError: name 'test_x' is not defined

In [28]:
"""Try another model"""
from keras.callbacks import TensorBoard, ModelCheckpoint
import model
# reload model module
import importlib
importlib.reload(model)

BATCH_SIZE = 128
EPOCHS = 4

model_checkpoint_callback = ModelCheckpoint(
    filepath='./checkpoints/model2',
    save_weights_only=False,  # Save the whole model
    monitor='val_loss',
    mode='min',
    save_best_only=True)

tensor_board_callback = TensorBoard(log_dir=fp.log_dir)

my_model = model.get_model2()
my_model.summary()
my_model.fit(
    train_x, 
    train_y, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    validation_data=(val_x, val_y),
    callbacks=[
        tensor_board_callback,
        model_checkpoint_callback
        ]
    )

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, 20)                2100      
                                                                 
 dense_6 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________


c:\Users\noah\miniconda3\envs\cse517\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/4
1306/1306 [==============================] - 45s 34ms/step - loss: 0.0504 - accuracy: 0.9824 - val_loss: 0.0023 - val_accuracy: 0.9994
Epoch 2/4
1306/1306 [==============================] - 46s 35ms/step - loss: 0.0037 - accuracy: 0.9990 - val_loss: 0.0027 - val_accuracy: 0.9992
Epoch 3/4
1306/1306 [==============================] - 47s 36ms/step - loss: 0.0024 - accuracy: 0.9993 - val_loss: 0.0011 - val_accuracy: 0.9998
Epoch 4/4
1306/1306 [==============================] - 45s 35ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 9.3487e-04 - val_accuracy: 0.9998


In [29]:
# Try model 3
import model
# reload model module
import importlib
importlib.reload(model)


BATCH_SIZE = 128
EPOCHS = 4

model_checkpoint_callback = ModelCheckpoint(
    filepath='./checkpoints/model3',
    save_weights_only=False,  # Save the whole model
    monitor='val_loss',
    mode='min',
    save_best_only=True)

tensor_board_callback = TensorBoard(log_dir=fp.log_dir)

my_model = model.get_model3()
my_model.summary()
my_model.fit(
    train_x, 
    train_y, 
    epochs=EPOCHS, 
    batch_size=BATCH_SIZE, 
    validation_data=(val_x, val_y),
    callbacks=[
        tensor_board_callback,
        model_checkpoint_callback
        ]
    )

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_normalization (LayerN  (None, 29, 13)           26        
 ormalization)                                                   
                                                                 
 gru_7 (GRU)                 (None, 20)                2100      
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,147
Trainable params: 2,147
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
1306/1306 [==============================] - 59s 43ms/step - loss: 0.0408 - accuracy: 0.9815 - val_loss: 0.0022 - val_accuracy: 0.9996
Epoch 2/4
1306/1306 [==============================] - 56s 43ms/step - loss: 0.0033 - accuracy: 0.9990 - 